In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('/content/moviereviews.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/moviereviews.csv'

In [ ]:
df.head()

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
lemmatizer = WordNetLemmatizer()
import string
nltk.download('wordnet')

In [ ]:
df['review'] = df['review'].astype(str)
def preprocess(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = nltk.word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
  return ' '.join(lemmatized_tokens)

In [ ]:
df['filtered_review'] = df['review'].apply(preprocess)

In [ ]:
df.head()

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
df['polarity_scores'] = df['filtered_review'].apply(sia.polarity_scores)

In [ ]:
df.head()

In [ ]:
import transformers
from transformers import pipeline

In [ ]:
df['filtered_review'] = df['filtered_review'].fillna('')
pipe = pipeline('sentiment-analysis')
df['sentiment'] = df['filtered_review'].apply(lambda x: pipe(x)[0]['label'])

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
matrix = cv.fit_transform(df[df['label'] == 'pos']['filtered_review'])

In [ ]:
freq_words = zip(cv.get_feature_names_out(),matrix.sum(axis=0).tolist()[0])
freq_words = sorted(freq_words,key=lambda x:x[1],reverse=True)[:20]

In [ ]:
freq_words

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB()

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
pipe = Pipeline([('tfidf',tfifidf),('model',model)])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df['filtered_review'],df['sentiment'],test_size=0.2,random_state=42)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
print(accuracy_score(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.tree import Tree

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')

In [ ]:
def extract_entities(text):
  tokens = nltk.word_tokenize(text)
  pos_tags = pos_tag(tokens)
  chunk = ne_chunk(pos_tags)
  entities = []

  for subtree in chunk:
    if isinstance(subtree,Tree):
      entity = ' '.join(token for token,pos in subtree.leaves())
      entity_type = subtree.label()
      entities.append((entity,entity_type))
  return entities

In [ ]:

df['entities'] = df['filtered_review'].apply(extract_entities)

In [ ]:
df.head()

In [ ]:
df['review'][0]

In [ ]:
pos_tags = pos_tag(nltk.word_tokenize(df['review'][0]))

In [ ]:
ne_chunk(pos_tags)

In [ ]:
!pip install svgling